In [ ]:
import numpy as np
import torch.cuda
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [1]:
def check_trainable(val,model):
    return filter(lambda p: p.requires_grad == val, model.parameters())

def print_params(generator):
    return list(map(lambda x: x.size(),generator))

def count_params(generator):
    return sum([np.prod(layer.size()) for layer in generator])

def get_trainable_parameters(model):
    #print(model)
    trainable = list(check_trainable(True,model))
    count_trainable = count_params(trainable)
    count_not_trainable = count_params(check_trainable(False,model))
    print("Trainable parameters:\t{:>12}\nFrozen parameters:\t{:>12}".format(count_trainable,count_not_trainable))
    return trainable

In [2]:
def varify(x, volatile=False, gpu_id=0):
    if isinstance(x, list):
        return [varify(y,volatile,gpu_id) for y in x]
    if torch.cuda.is_available():
        #x = x.cuda(async=True)
        x = x.cuda(gpu_id)
    return Variable(x, volatile=volatile)

In [3]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [4]:
def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res